# Test hypotéz
## Konanie testov dvoch zadaných hypotéz nad dátami týkajúcich sa nehôd v ČR

In [16]:
import pandas as pd
from scipy.stats import chi2_contingency, ttest_ind

In [17]:
data = pd.read_pickle("accidents.pkl.gz")
data.head()

,p1,p36,p37,p2a,weekday(p2a),p2b,p6,p7,p8,p9,...,n,o,p,q,r,s,t,p5a,region,date
0,2100160001,4,NaN,2016-01-01,5,55,1,1,0,2,...,711403.0,NaN,Souhlasnýsesměremúseku,Pomalý,554782.0,451622.0,GN_V0.1UIR-ADR_410,1,PHA,2016-01-01
1,2100160002,4,NaN,2016-01-01,5,130,1,3,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,PHA,2016-01-01
2,2100160003,5,NaN,2016-01-01,5,100,1,2,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,PHA,2016-01-01
3,2100160004,6,NaN,2016-01-01,5,120,9,0,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,PHA,2016-01-01
4,2100160005,6,NaN,2016-01-01,5,2560,2,0,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,PHA,2016-01-01


## Hypotéza 1 - test s istotou 95%
### Na cestách prvej triedy sa pri nehodách umieralo s rovnakou pravdepodobnosťou ako na cestách tretej triedy

Hypotézu otestujem testom $\chi^2$

Na test budem potrebovať z dát vybrať stĺpce, ktoré vyznačujú:
* Druh komunikácie = dátová položka 'p36', 
  * kde cesty prvej triedy sú označené hodnotou '1' 
  * a cesty tretej triedy sú označené hodnotou '3'
* Počet mŕtvych osôb = dátová položka 'p13a'

In [18]:
h1 = data.copy()

h1 = h1[['p13a', 'p36']]

# Limit the dataset to only relevant types of roads
h1 = h1[h1['p36'].isin([1, 3])].reset_index(drop=True)

Ďalej, pre určenie binárnych dát pre test hypotézy $\chi^2$ vytvorím ďalšie dva stĺpce:
* Fatálna nehoda - Rozdeľuje nehody na fatálne (niekto pri nehode umrel) a nefatálne
* Trieda cesty - Rozdeľuje nehody podľa toho, či sa stali na ceste 1. triedy, alebo na ceste 3. triedy

In [19]:
h1['fatalna nehoda'] = h1['p13a'] > 0
h1['fatalna nehoda'] = h1['fatalna nehoda'].map({False: "NIE", True: "ANO"})

h1.loc[h1['p36'] == 1, 'trieda cesty'] = "1."
h1.loc[h1['p36'] == 3, 'trieda cesty'] = "3."        
 
h1.head()

,p13a,p36,fatalna nehoda,trieda cesty
0,0,1,NIE,1.
1,0,1,NIE,1.
2,0,1,NIE,1.
3,0,1,NIE,1.
4,0,1,NIE,1.


Transformovanie dát do vhodného formátu pre prevedenie testu hypotézy $\chi^2$

In [20]:
ct = pd.crosstab(h1['fatalna nehoda'], h1['trieda cesty'])
ct

trieda cesty,1.,3.
fatalna nehoda,,
ANO,1104,536
NIE,96618,91882


Prevedenie testu hypotézy $\chi^2$

In [21]:
statistic, pvalue, dof, expected = chi2_contingency(ct)

print(pvalue)
print(f"Hypotéza zamietnutá: {pvalue < 0.05}")
display(expected)

2.9583564622976284e-38
Hypotéza zamietnutá: True


array([[  842.87409277,   797.12590723],
       [96879.12590723, 91620.87409277]])

Mimo samotného výsledku testu hypotézy ešte zisťujem aj skutočnosť, 
či viedli nehody na cestách 1. triedy častejšie alebo menej často k fatálnym nehodám oproti cestám 3. triedy

In [22]:
expected_fatalities_1stclass = expected[0][0] # Trieda cesty == '1.' && Fatalna nehoda == 'ANO' 
expected_fatalities_3rdclass = expected[0][1] # Trieda cesty == '3.' && Fatalna nehoda == 'ANO'  
expected_fatalities_sum = expected_fatalities_1stclass + expected_fatalities_3rdclass

fatality_rate_1stclass = expected_fatalities_1stclass / expected_fatalities_sum  
fatality_rate_3rdclass = expected_fatalities_3rdclass / expected_fatalities_sum

print(f"Očakávaná pravdepodobnosť úmrtia pri zrážke na ceste 1. triedy: {fatality_rate_1stclass * 100}")
print(f"Očakávaná pravdepodobnosť úmrtia pri zrážke na ceste 3. triedy: {fatality_rate_3rdclass * 100}")
print()
print(f"Nehody na cestách 1. triedy vedú k fatálnym nehodám pravdepodobnejšie: "
        f"{fatality_rate_1stclass > fatality_rate_3rdclass}")

Očakávaná pravdepodobnosť úmrtia pri zrážke na ceste 1. triedy: 51.394761754496685
Očakávaná pravdepodobnosť úmrtia pri zrážke na ceste 3. triedy: 48.605238245503315

Nehody na cestách 1. triedy vedú k fatálnym nehodám pravdepodobnejšie: True


## Hypotéza 1 - Záver:
---------------------- 

$H_0$ zamietam a prímam alternatívnu hypotézu:    
#### Na cestách prvej triedy sa pri nehodách NEUMIERALO s rovnakou pravdepodobnosťou ako na cestách tretej triedy
#### Zistil som taktiež, že nehody na cestách 1. triedy vedú k fatálnym nehodám pravdepodobnejšie.

## Hypotéza 2 - test s istotou 95%
### Pri nehodách vozidiel značky ŠKODA je škoda na vozidle nižšia ako pri nehodách vozidiel AUDI

Na test Hypotézy 2 budem potrebovať z dát vybrať stĺpce, ktoré vyznačujú:
* Značka vozidla = dátová položka 'p45a', 
  * kde vozidlá značky ŠKODA sú označené hodnotou '39' 
  * a vozidlá značky AUDI sú označené hodnotou '2'
* Škoda na vozidle = dátová položka 'p53'

In [23]:
h2 = data.copy()

h2 = h2[['p45a', 'p14']] 
# Limit the dataset to only relevant car brands
h2 = h2[h2['p45a'].isin([2, 39])].reset_index(drop=True)

h2.head()

,p45a,p14
0,2.0,4800
1,39.0,200
2,39.0,500
3,39.0,50
4,39.0,1250


Ďalej, z vytvoreného datasetu vezmem separátne škody na vozidlách podľa ich značky.
Už priemery škôd z dát pre AUDI a ŠKODA napovedajú, že vozidlá ŠKODA v priemere utrpia nižšie škody.

In [24]:
damages_audi = h2[h2['p45a'] == 2]['p14']
damages_skoda = h2[h2['p45a'] == 39]['p14']

print(damages_audi.mean(), damages_skoda.mean())
print(damages_audi.std(), damages_skoda.std())

924.7309454981315 683.0897848470563
1397.043006289386 2150.260899179228


Pre otestovanie hypotézy 2. použijem nezávislý párový t-test (implementácia modulom scipy.stat -> ttest_ind(a, b))

In [30]:
result = ttest_ind(damages_skoda, damages_audi, equal_var=False) 
print(result)

TtestResult(statistic=-18.576738138973823, pvalue=1.9879833820234112e-76, df=21331.839351629238)


A na koniec skontrolujem či je $H_0$ zamietnutá (kontrola štatistickej významnosti):

In [26]:
print(f"Hypotéza zamietnutá: {result.pvalue < 0.05 and result.statistic > 0}")

Hypotéza zamietnutá: False


## Hypotéza 2 - Záver: 
----------------------
### $H_0$ nezamietam:    
#### Pri nehodách vozidiel značky ŠKODA je škoda na vozidle NIŽŠIA ako pri nehodách vozidiel AUDI